In [1]:
import os
import math

import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import AxesZero
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
import matplotlib.ticker as mtick
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
from scipy.stats import poisson, norm, lognorm, gamma
from scipy import optimize as opti
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.signal import savgol_filter
import h5py
import torch
import cupy as cp

import wf_func as wff

np.random.seed(0)

In [2]:
Mu = 4
Tau = 20
Sigma = 5
file = '4.0-20-5'

In [3]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]

In [4]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=gmu, scale=gsigma)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [5]:
Thres = wff.Thres
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
p = spe_pre[0]['parameters']
window = wff.window
t_auto = np.arange(window).reshape(window, 1) - np.arange(window).reshape(1, window)
mnecpu = wff.spe((t_auto + np.abs(t_auto)) / 2, p[0], p[1], p[2])

In [6]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.annotate('', 
            xy=(17, 0.07), 
            xytext=(17, 0.04), 
            arrowprops=dict(facecolor='k', arrowstyle='<|-|>'))
ax.text(x=20, y=0.055, s=r'$\mu$')
ax.annotate('', 
            xy=(-13, 0.09), 
            xytext=(13, 0.09), 
            arrowprops=dict(facecolor='k', arrowstyle='<|-|>'))
ax.text(x=0, y=0.093, s=r'$t_0$')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\textrm{probability density}$')
ax.legend(title=r'$(\tau_\ell, \sigma_\ell)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.2)
# ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()
ax.get_position()

Bbox([[0.125, 0.12], [0.9, 0.89]])

In [7]:
fig = plt.figure(figsize=(8, 4))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.05, right=0.97, top=0.97, bottom=0.1, wspace=0.3, hspace=0.3)
ax = fig.add_subplot(gs[0, 0])

ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.plot(1, 0, '>k', transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot(0, 1, '^k', transform=ax.get_xaxis_transform(), clip_on=False)

t = np.linspace(0, 6, 201)
ax.plot(t, lognorm.pdf(t, loc=0, s=0.3), color='darkorange')
ax.plot(t, lognorm.pdf(t, loc=3, s=0.3), color='darkblue')
ax.fill_between(t, 0, lognorm.pdf(t, loc=0, s=0.3), color='darkorange', alpha=0.5)
ax.fill_between(t, 0, lognorm.pdf(t, loc=3, s=0.3), color='darkblue', alpha=0.5)
ax.set_xlim(0, 6)
ax.set_ylim(bottom=1e-3)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r'$\mathrm{Time}$')
ax.set_ylabel(r'$\mathrm{Voltage}$')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.annotate(text='', xy=(1.5, 1), xytext=(3.5, 1), arrowprops=dict(arrowstyle='<->'))
ax.text(x=2.2, y=1.1, s=r'$\sim D_\mathrm{w}$')
ax.text(x=0.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
ax.text(x=3.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[cid]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,2,0,469.16,96.56
1,2,0,471.43,200.71
2,2,0,487.39,220.02
3,2,0,499.50,165.30
4,2,0,527.69,188.54


In [9]:
ax1_lim = [min(0, wave.min() * 1.2), wave.max() * 1.3]
ax2_lim = [ax1_lim[0] * 1.7 /  ax1_lim[-1], 1.7]

In [10]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 200)

In [11]:
TRIALS = 5000

#profile
def vcombine(A, cx, t, w_all):
    '''
    t is 2 x l_e
    '''
    frac, ti = cp.modf(t)
    ti = cp.array(ti, np.int32)
    A_vec = (1 - frac)[:, :, None] * A[w_all, :, ti] + frac[:, :, None] * A[w_all, :, ti+1]
    c_vec = (1 - frac)[:, :, None] * cx[w_all, :, ti] + frac[:, :, None] * cx[w_all, :, ti+1]
    return A_vec, c_vec

vstep = cp.array((-1, 1), np.float32)

#profile
def vmove1(A_vec, c_vec, z, fmu, fsig2s_inv, det_fsig2s_inv, b0):
    '''
    A_vec: 行向量 x 2
    c_vec: 行向量 x 2
    vstep: 构造 (-1, 1) 则 np.einsum('ej,ejk,ek->e', fmu, fsig2s_inv, fmu) 恒等于 0

    cp.einsum('eiw,ejw->eij', A_vec, c_vec)

    cp.einsum('ejk,ek->ej', c_vec, z) + cp.einsum('ej,ejk->ek', fmu, fsig2s_inv)
    cp.einsum('ej,ejk,ek->e', zc, beta, zc)
    '''
    ac = A_vec @ cp.transpose(c_vec, (0, 2, 1))
    beta_inv = fsig2s_inv + (ac + cp.transpose(ac, (0, 2, 1))) / 2
    beta = cp.linalg.inv(beta_inv)

    zc = (c_vec @ z[:, :, None]).squeeze() + (fmu[:, None, :] @ fsig2s_inv).squeeze()
    Δν = 0.5 * (zc[:, None, :] @ beta @ zc[:, :, None]).squeeze()

    Δν += 0.5 * cp.log(cp.clip(cp.linalg.det(beta) * det_fsig2s_inv, 1/b0, b0))
    return Δν, beta

#profile
def vmove2(A_vec, c_vec, fmu, A, beta):
    '''
    "eij, eiv, ejw, ewt->evt"
    beta c_vec c_vec A
    '''
    Δcx = -cp.transpose(beta @ c_vec, (0, 2, 1)) @ (c_vec @ A)
    Δz = -(fmu[:, None, :] @ A_vec).squeeze()
    return Δcx, Δz

def periodic(_h, _lt):
    '''
    enforce a periodic boundary at 0 and _lt - 1
    '''
    c_overflow = _h < 0
    _h[c_overflow] += _lt[c_overflow] - 1
    c_overflow = _h > _lt - 1
    _h[c_overflow] -= _lt[c_overflow] - 1
    return _h

#profile
def v_rt(_s, _ts, w_all):
    frac, ti = np.modf(_s)
    ti = np.array(ti, np.int32)
    return (1 - frac) * _ts[w_all, ti] + frac * _ts[w_all, ti+1]
    
tau = Tau
sigma = Sigma

pi = np.pi
b_t0 = [0., 600.]

# def lc(t):
#     return co + np.log(1.0 - erf((ass - t)/s2s)) - alpha*t

def lc(t):
    return wff.log_convolve_exp_norm(t, tau, sigma)

if tau == 0.0:
    sel_lc = cp.ElementwiseKernel(
        'float32 t, bool sel',
        'float32 lprob',
        f"lprob = sel ? -log({sigma}) - 0.5 * log(2.0 * {pi}) - 0.5 * (t / {sigma}) * (t / {sigma}) : 0",
        'sel_lc')
elif sigma == 0.0:
    sel_lc = cp.ElementwiseKernel(
        'float32 t, bool sel',
        'float32 lprob',
        f"lprob = sel ? (t > 0 ? -log({tau}) - t / {tau} : -1e4) : 0",
        'sel_lc')
else:
    alpha = 1 / tau
    co = -np.log(2.0 * tau) + alpha * alpha * sigma * sigma / 2.0
    ass = alpha * sigma * sigma
    s2s = np.sqrt(2.0) * sigma
    sel_lc = cp.ElementwiseKernel(
        'float32 t, bool sel',
        'float32 lprob',
        f"lprob = sel ? {co} + log(1.0 - erf(({ass} - t) / {s2s})) - {alpha} * t : 0",
        'sel_lc')

sel_add = cp.ElementwiseKernel('float32 delta, bool sel', 'float32 dest', "if(sel) dest += delta", "sel_add")
sel_assign = cp.ElementwiseKernel('float32 value, bool sel', 'float32 dest', "if(sel) dest = value", "sel_assign")

#profile
def batch(A, cx, index, tq, s, z, t0_min=100, t0_max=500):
    """
    batch
    ====
    连续时间游走
    cx: Cov^-1 * A, 详见 FBMP
    s: list of PE locations
    mu_t: LucyDDM 的估算 PE 数
    z: residue waveform (raw - predicted)

    home_s 是在指标意义上的连续变量

    0   1   2   3   4   5   6   7   8   9   l_t = 10
    +---+---+---+---+---+---+---+---+---+
    0  0.1 0.2 0.2 0.5 0.4 0.3 0.2  0   0   q_s
    0  0.1 0.3 0.5 1.0 1.4 1.7 1.9 1.9 1.9  cq
     _h \in [0, 9) 范围的测度是 9

    int(_h) 是插值的下指标 占比为 1 - decimal(_h)
    int(_h) + 1 是插值的上指标 占比为 decimal(_h)
    """
    sig2w = cp.asarray(index["sig2w"], np.float32)
    sig2s = cp.asarray(index["sig2s"], np.float32)
    mus = cp.asarray(index["mus"], np.float32)
    NPE = index["NPE"]

    a0 = A[:, :, 0]
    b0 = 1 + sig2s * (a0[:, None, :] @ a0[:, :, None]).squeeze() / sig2w

    l_e = len(index)
    l_s = s.shape[1]
    w_all = cp.arange(l_e) # index of all the waveforms

    l_t = tq.shape[1]
    # istar [0, 1) 之间的随机数，用于点中 PE
    istar = np.random.rand(l_e, TRIALS) # 同时可用于创生位置的选取

    # 根据 p_cha 采样得到的 PE 序列。供以后的产生过程使用。这两行是使用了 InverseCDF 算法进行的MC采样。
    fp = np.arange(l_t)
    home_s = np.array([np.interp(_is, xp=_xp[:_lt], fp=fp[:_lt]) for _is, _xp, _lt in zip(istar, tq["cq"], index["l_t"])])

    t0 = np.zeros(l_e, np.float32)
    e_hit = NPE > 0

    t0[e_hit] = v_rt(s[e_hit, 0], tq["t_s"], e_hit)
    e_nonhit = ~e_hit
    t0[e_nonhit] = tq["t_s"][e_nonhit, 0]
    t0 = np.clip(t0, t0_min, t0_max)

    # step: +1 创生一个 PE， -1 消灭一个 PE， +2 向左或向右移动
    flip = np.random.choice((-1, 1, 2), (l_e, TRIALS), p=np.array((1, 1, 2)) / 4)
    Δν_g = cp.zeros(l_e, dtype=np.float32)
    Δν = np.zeros(l_e, dtype=np.float32)
    ν = np.zeros(l_e, dtype=np.float32)
    ν_max = np.zeros(l_e, dtype=np.float32)
    Δν_history = np.zeros((l_e, TRIALS), dtype=np.float32) # list of Δν's
    annihilations = np.zeros((l_e, TRIALS)) # float64
    creations = np.zeros((l_e, TRIALS)) # float64
    t0_history = np.zeros((l_e, TRIALS), dtype=np.float32)
    s0_history = np.zeros((l_e, TRIALS), dtype=np.uint32) # 0-norm of s
    s_history = np.zeros((l_e, TRIALS * l_s), dtype=np.float32)

    # t0_accept = 0

    log_mu = np.log(index["mu0"])  # 猜测的 Poisson 流强度
    loc = np.zeros((l_e, 2)) # float64
    fsig2s_inv = cp.diag(vstep)[None, :, :] * (1/sig2s)[:, None, None]
    det_fsig2s_inv = cp.linalg.det(fsig2s_inv)
    fmu = vstep[None, :] * mus[:, None]

    last_max_s = s.copy()
    s_max_index = np.zeros(l_e, dtype=np.uint32)

    for i, (t, step, home, wander, wt, accept, acct) in enumerate(
        zip(
            istar.T,
            flip.T,
            home_s.T,
            np.random.normal(size=(TRIALS, l_e)),
            np.random.normal(scale=10, size=(TRIALS, l_e)),
            *np.log(np.random.rand(2, TRIALS, l_e)),
        )
    ):
        if i % 1000 == 0:
            print(i)

        mNPE = np.max(NPE)
        rt = v_rt(s[:, :mNPE], tq["t_s"], np.arange(l_e)[:, None])
        nt0 = t0 + wt.astype(np.float32)
        sel = cp.asarray(np.arange(mNPE)[None, :] < NPE[:, None])
        lc0 = cp.sum(sel_lc(cp.asarray(rt - t0[:, None]), sel), axis=1)
        lc1 = cp.sum(sel_lc(cp.asarray(rt - nt0[:, None]), sel), axis=1)
        t0_acceptance = np.logical_and(cp.asnumpy(lc1 - lc0) >= acct, np.logical_and(nt0 >= t0_min, nt0 <= t0_max))
        # t0_accept += np.sum(t0_acceptance) / len(t0_acceptance)
        np.putmask(t0, t0_acceptance, nt0)
        t0_history[:, i] = t0

        ### 光变曲线和移动计算
        Δν[:] = 0
        e_bounce = NPE == 0
        step[e_bounce] = 1
        accept[e_bounce] += np.log(4)  # 惩罚
        ea_bounce = np.logical_and(NPE == 1, step == -1)
        accept[ea_bounce] -= np.log(4) # 1 -> 0: 行动后从 0 脱出的几率大，需要鼓励

        e_create = step == 1
        e_minus = ~e_create
        e_move = step == 2
        e_pm = ~e_move
        e_annihilate = step == -1
        e_plus = ~e_annihilate
        loc[e_create, 0] = l_t # 0 A_vec
        op = np.array(t * NPE, dtype=np.int32)
        loc[e_minus, 0] = s[e_minus, op[e_minus]] # annihilate + move
        loc[e_move, 1] = periodic(loc[e_move, 0] + wander[e_move], index["l_t"][e_move])
        loc[e_create, 1] = periodic(home[e_create], index["l_t"][e_create])
        loc[e_annihilate, 1] = l_t

        ### 矩阵 Δν 计算
        vA, vc = vcombine(A, cx, cp.asarray(loc, np.float32), w_all[:, None])
        Δν_g, beta = vmove1(vA, vc, z, fmu, fsig2s_inv, det_fsig2s_inv, b0)

        ## -1 cases, step == 2, -1
        Δν[e_minus] -= lc(v_rt(loc[e_minus, 0], tq["t_s"], e_minus) - t0[e_minus])
        ## +1 cases, step == 2, 1
        Δν[e_plus] += lc(v_rt(loc[e_plus, 1], tq["t_s"], e_plus) - t0[e_plus])
        ## non-move cases, step == 1, -1
        NPE[e_create] += 1
        loc[e_annihilate, 1] = loc[e_annihilate, 0]
        Δν[e_pm] += step[e_pm] * (log_mu[e_pm] - np.log(tq["q_s"][e_pm, np.array(loc[e_pm, 1], dtype=np.int32) + 1]) - np.log(NPE[e_pm]))
        loc[e_annihilate, 1] = l_t
        NPE[e_create] -= 1
        ########
        Δν += cp.asnumpy(Δν_g)
        #######

        ### 计算 Δcx, Δz, 更新 cx 和 z。对 accept 进行特别处理
        e_accept = Δν >= accept
        _e_accept = cp.asarray(e_accept)
        Δcx, Δz = vmove2(vA, vc, fmu, A, beta)
        sel_add(Δcx, _e_accept[:, None, None], cx)
        sel_add(Δz, _e_accept[:, None], z)
        ########

        # 增加
        ea_create = np.logical_and(e_accept, e_create)
        ea_plus = np.logical_and(e_accept, e_plus)
        creations[ea_plus, i] = loc[ea_plus, 1]

        s[ea_create, NPE[ea_create]] = loc[ea_create, 1]
        NPE[ea_create] += 1
        # 减少
        ea_annihilate = np.logical_and(e_accept, step == -1)
        ea_minus = np.logical_and(e_accept, e_minus)
        annihilations[ea_minus, i] = loc[ea_minus, 0]

        NPE[ea_annihilate] -= 1
        s[ea_annihilate, op[ea_annihilate]] = s[ea_annihilate, NPE[ea_annihilate]]
        # 移动
        ea_move = np.logical_and(e_accept, e_move)
        s[ea_move, op[ea_move]] = loc[ea_move, 1]

        Δν[~e_accept] = 0
        step[~e_accept] = 0

        ν += Δν
        breakthrough = ν > ν_max
        ν_max[breakthrough] = ν[breakthrough]
        s_max_index[breakthrough] = i
        last_max_s[breakthrough] = s[breakthrough]
        Δν_history[:, i] = Δν
        flip[:, i] = step
        s0_history[:, i] = NPE
        s_history[:, i*l_s:(i+1)*l_s] = s
    # print("t0 acceptance:", t0_accept / TRIALS)
    # Assign one PE to the waveform without any PE
    s_init = np.full(l_s, 0.)
    s_init[0] = 1.
    last_max_s[last_max_s.sum(axis=1) == 0] = s_init
    return flip, s0_history, t0_history, Δν_history, s_max_index, last_max_s, annihilations, creations, s_history

def get_t0(a0, a1, s0_history, t0_history, loc, flip, index, tq, t00_l):
    l_e = a1 - a0
    l_s = loc.shape[1] // TRIALS
    t0_l = np.empty(l_e)
    mu_l = np.empty(l_e)
    for i in range(a0, a1):
        # accept = flip[i] != 0
        accept = np.full(len(flip[i]), True)
        NPE = s0_history[i]
        number_sample_zero = np.sum(NPE == 0)
        t00_list = np.repeat(t0_history[i][accept], NPE[accept])
        step = np.repeat(np.arange(TRIALS)[accept], NPE[accept])
        idx_base = np.arange(TRIALS)[accept] * l_s
        idx = np.hstack([i_b + np.arange(npe) for i_b, npe in zip(idx_base, NPE[accept])])
        mu_t = index["mu0"][i]
        b_mu = [max(1e-8, mu_t - 5 * np.sqrt(mu_t)), mu_t + 5 * np.sqrt(mu_t)]
        l_t = index["l_t"][i]
        loc_i = loc[i][idx]
        ilp_cha = np.log(1 / tq["q_s"][i][:l_t])
        guess = ilp_cha[loc[i][idx].astype(int)]
        loc_i = np.interp(loc_i, xp=np.arange(0.5, l_t), fp=tq["t_s"][i][:l_t])
        # t00 = index["t0"][i]
        # t00 = loc_i.mean() + 1
        t00 = t00_l[i]
        t0_l[i - a0], mu_l[i - a0] = wff.fit_t0mu_gibbs(loc_i, t00_list, step, number_sample_zero, tau, sigma, mu_t, t00, b_mu, b_t0, TRIALS)
    return t0_l, mu_l

In [12]:
with h5py.File('result/fsmp_test/sparsify/' + file + '.h5', "r", libver="latest", swmr=True) as ipt:
    A = ipt["A"][:]
    cx = ipt["cx"][:]
    index = ipt["index"][:]
    s = ipt["s"][:]
    tq = ipt["tq"][:]
    z = ipt["z"][:]

In [13]:
# Try using Metropolis
# TRIALS = 5000
# n = 1
# b_t0 = [0., 600.]

# # initialization
# A, y, tlist, t0_t, t0_delta, cha, left_wave, right_wave = wff.initial_params(wave[::wff.nshannon], spe_pre[cid], Tau, Sigma, gmu, Thres['lucyddm'], p, is_t0=True, is_delta=False, n=n)
# # assert len(np.unique(np.diff(tlist))) == 1
# s_cha = np.cumsum(cha)
# # moving average filter of size 2*n+1
# cha = np.pad(s_cha[2*n+1:], (n+1, n), 'edge') - np.pad(s_cha[:-(2*n+1)], (n+1, n), 'edge')
# cha += 1e-8 # for completeness of the random walk.
# p_cha = cha / np.sum(cha)
# mu_t = abs(y.sum() / gmu)

# # Eq. (9) where the columns of A are taken to be unit-norm.
# mus = np.sqrt(np.diag(np.matmul(A.T, A)))
# assert np.std(mus) < 1e-4, 'mus must be equal'
# mus = mus[0]
# A = A / mus
# p1 = mu_t * wff.convolve_exp_norm(tlist - t0_t, Tau, Sigma) / n + 1e-8
# sig2w = spe_pre[cid]['std'] ** 2
# sig2s = (gsigma * mus / gmu) ** 2

# nu_star, T_star, c_star, es_history, NPE_evo, number_sample_zero = wff.metropolis_fsmp(y, A, sig2w, sig2s, mus, p1, p_cha, mu_t)

# ilp_cha = np.log(cha.sum()) - np.log(cha)
# guess = ilp_cha[es_history['loc'].astype(int)]
# es_history['loc'] = np.interp(es_history['loc'], xp=np.arange(0.5, len(tlist)), fp=tlist)
# ans = opti.fmin_l_bfgs_b(lambda x: -np.sum(wff.log_convolve_exp_norm(es_history['loc'] - x, Tau, Sigma)), x0=[t0_t], approx_grad=True, bounds=[b_t0], maxfun=500000)
# t00 = ans[0].item() if ans[-1]['warnflag'] == 0 else t0_t
# b_mu = [max(1e-8, mu_t - 5 * np.sqrt(mu_t)), mu_t + 5 * np.sqrt(mu_t)]
# t0, mu = wff.fit_t0mu_guess(es_history['loc'], es_history['step'], 
#                             number_sample_zero, Tau, Sigma, guess, mu_t, t00, b_mu, b_t0, TRIALS)

# j = 0
# xmmse_most = np.zeros(len(tlist))
# while np.all(xmmse_most <= 0):
#     maxindex = nu_star.argsort()[::-1][j]
#     zx = y - np.dot(A, mus * c_star[maxindex])
#     Phi_s = wff.Phi(y, A, c_star[maxindex], mus, sig2s, sig2w)
#     invPhi = np.linalg.inv(Phi_s)
#     xmmse_most = mus * c_star[maxindex] + np.matmul(np.diagflat(sig2s * c_star[maxindex]), np.matmul(A.T, np.matmul(invPhi, zx)))
#     j += 0
# pet = np.repeat(tlist[xmmse_most > 0], c_star[maxindex][xmmse_most > 0])
# cha = np.repeat(xmmse_most[xmmse_most > 0] / mus / c_star[maxindex][xmmse_most > 0], c_star[maxindex][xmmse_most > 0])

# Trying to use wangyy's file
# with h5py.File('result/fsmp/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
#     photoelectron = ipt['photoelectron'][:]

# pet = photoelectron['HitPosInWindow'][(photoelectron['TriggerNo'] == eid) & (photoelectron['ChannelID'] == cid)]
# cha = photoelectron['Charge'][(photoelectron['TriggerNo'] == eid) & (photoelectron['ChannelID'] == cid)] / gmu

# with h5py.File('result/fsmp/solu/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
#     time = ipt['starttime'][:]

# t0 = time['tswave'][(time['TriggerNo'] == eid) & (time['ChannelID'] == cid)]
# mu = time['muwave'][(time['TriggerNo'] == eid) & (time['ChannelID'] == cid)]

l_e = len(index)
s_t = np.argsort(index["l_t"])

i_part = np.array([i, i+1])
l_part = len(i_part)
ind_part = index[i_part]
lp_t = np.max(ind_part["l_t"])
lp_wave = np.max(ind_part["l_wave"])
lp_NPE = np.max(ind_part["NPE"])
print(lp_t, lp_wave, lp_NPE)

null = np.zeros((l_part, lp_wave, 1), np.float32) # cx, A[:, :, -1] = 0  用于 +- 的空白维度
s_null = np.zeros((l_part, lp_NPE * 2), np.float32) # 富余的 PE 活动空间
(flip, s0_history, t0_history, Δν_history, s_max_index, last_max_s, annihilations, creations, loc
 ) = batch(cp.asarray(np.append(A[i_part, :lp_wave, :lp_t], null, axis=2), np.float32),
           cp.asarray(np.append(cx[i_part, :lp_wave, :lp_t], null, axis=2), np.float32),
           index[i_part], 
           tq[i_part, :lp_t], 
           np.append(s[i_part, :lp_NPE], s_null, axis=1),
           cp.asarray(z[i_part, :lp_wave], np.float32))

def get_t0(a0, a1, s0_history, t0_history, loc, flip, index, tq, t00_l):
    l_e = a1 - a0
    l_s = loc.shape[1] // TRIALS
    t0_l = np.empty(l_e)
    mu_l = np.empty(l_e)
    for i in range(a0, a1):
        # accept = flip[i] != 0
        accept = np.full(len(flip[i]), True)
        NPE = s0_history[i]
        number_sample_zero = np.sum(NPE == 0)
        t00_list = np.repeat(t0_history[i][accept], NPE[accept])
        step = np.repeat(np.arange(TRIALS)[accept], NPE[accept])
        idx_base = np.arange(TRIALS)[accept] * l_s
        idx = np.hstack([i_b + np.arange(npe) for i_b, npe in zip(idx_base, NPE[accept])])
        mu_t = index["mu0"][i]
        b_mu = [max(1e-8, mu_t - 5 * np.sqrt(mu_t)), mu_t + 5 * np.sqrt(mu_t)]
        l_t = index["l_t"][i]
        loc_i = loc[i][idx]
        ilp_cha = np.log(1 / tq["q_s"][i][:l_t])
        guess = ilp_cha[loc[i][idx].astype(int)]
        loc_i = np.interp(loc_i, xp=np.arange(0.5, l_t), fp=tq["t_s"][i][:l_t])
        # t00 = index["t0"][i]
        # t00 = loc_i.mean() + 1
        t00 = t00_l[i]
        t0_l[i - a0], mu_l[i - a0] = wff.fit_t0mu_gibbs(loc_i, t00_list, step, number_sample_zero, tau, sigma, mu_t, t00, b_mu, b_t0, TRIALS)
    return t0_l, mu_l

42 173 6
0
1000
2000
3000
4000


In [14]:
l_t = index['l_t'][i]
tlist = tq['t_s'][i][:l_t]
mus = index['mus'][i]
sig2s = index['sig2s'][i]
sig2w = index['sig2w'][i]
y = ent[i]['Waveform'][index['a_wave'][i]:index['b_wave'][i]]
A_i = A[i][:index['b_wave'][i]-index['a_wave'][i], :l_t]

s_index = s0_history.flatten()[s_max_index + np.arange(2) * TRIALS]

s = last_max_s[0][:s_index[0]]
t, c = np.unique(np.sort(np.digitize(s, bins=np.arange(l_t)) - 1), return_counts=True)
c_star = np.zeros(l_t, dtype=int)
c_star[t] = c

zx = y - np.dot(A_i, mus * c_star)
Phi_s = wff.Phi(y, A_i, c_star, mus, sig2s, sig2w)
invPhi = np.linalg.inv(Phi_s)
xmmse_most = mus * c_star + np.matmul(np.diagflat(sig2s * c_star), np.matmul(A_i.T, np.matmul(invPhi, zx)))
pet = np.repeat(tlist[xmmse_most > 0], c_star[xmmse_most > 0])
cha = np.repeat(xmmse_most[xmmse_most > 0] / mus / c_star[xmmse_most > 0], c_star[xmmse_most > 0])
mu_i = (c_star > 0).sum()
t0_i, _ = wff.likelihoodt0(pet, char=cha, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='all')
pet, cha = wff.clip(pet, cha, 0.0)
# cha = cha * gmu

In [15]:
t0, mu = get_t0(0, 1, s0_history, t0_history, loc, flip, index[i_part], tq[i_part, :lp_t], t0truth['T0'][i_part])

pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
wave_ylim = ax.get_ylim()
fig.savefig('Note/figures/fsmp.pgf')
fig.savefig('Note/figures/fsmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [469. 471. 487. 499. 527.], Weight = [0.33516652 1.51841157 1.3589372  1.03511034 1.14905059]
wdist = 0.692857223254407, cdiff = -7.673615249627758
RSS = 59.251812237554844
1.935656595114267


In [16]:
# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.plot(wave, label='Waveform')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
# ax.set_xlim(0, len(wave))
# wave_ylim = ax.get_ylim()
# ax.set_ylim(wave_ylim[0] * 1.05, wave_ylim[1] * 1.05)
# ax.legend()
# fig.savefig('Note/figures/wave.pgf')
# fig.savefig('Note/figures/wave.pdf')
# fig.savefig('Note/figures/wave.png')
# fig.clf()
# plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge', linewidth=1.0)
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax2.set_ylim(wave_ylim[0] / 30, wave_ylim[1] / 30)
ax.plot(wave, label='Waveform')
ax.set_xlim(xmin, xmax)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.savefig('Note/figures/wave.png')
fig.clf()
plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
fig.savefig('Note/figures/spe.png')
plt.close()

# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Charge}$')
# ax.set_xlim(0, len(wave))
# ax.set_ylim(wave_ylim[0] / 20, wave_ylim[1] / 20)
# ax.axhline(y=0, color='k', linestyle='dashed', alpha=0.5)
# ax.legend()
# fig.savefig('Note/figures/charge.pgf')
# fig.savefig('Note/figures/charge.pdf')
# fig.savefig('Note/figures/charge.png')
# fig.clf()
# plt.close(fig)

In [17]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [18]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise

869.1974059198186
871.1250094058131


In [19]:
t = np.load('result/takara/char/Channel00/cnn_testing_record_2022-04-09_06:20:24.npz')['arr_0']
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax.plot(np.arange(1, len(t)+1), t, label=r'$D_\mathrm{w}$', color='C1')
ax.set_xlabel(r'$\mathrm{epoch}$')
ax.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [20]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481
 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499
 500 501 502 503 504 505 506 507 522 523 524 525 526 527 528 529 530 531
 532 533 534 535], Weight = [0.04728835 0.0773302  0.12971889 0.14296212 0.17259763 0.18196951
 0.16468631 0.16345413 0.15766181 0.12150366 0.10900383 0.10819887
 0.08709687 0.05954987 0.05300041 0.04706557 0.04386505 0.05970238
 0.10148353 0.11900193 0.14771352 0.15423084 0.1459499  0.13409241
 0.12377091 0.11172638 0.10319991 0.08280433 0.08618624 0.07160304
 0.10946468 0.10993917 0.13333807 0.12401741 0.12562733 0.11987713
 0.10082539 0.09262077 0.07290046 0.06250504 0.05116336 0.03533935
 0.04415651 0.04521806 0.05780158 0.07376266 0.10725069 0.10430699
 

In [21]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
assert ax1_lim[-1] > amp.max() + 10
for j in range(len(loc)):
    ax.annotate('', xy=(loc[j], amp[j]+5), xytext=(loc[j], amp[j]+10), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [472 489 500 529], Weight = [1.60835669 1.30957107 1.22689949 1.13247991]
wdist = 2.35038617939221, cdiff = -26.77229140635238
RSS = 739.9283743731916
4.852010259042913


In [22]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [467 468 469 470 471 472 473 474 475 484 485 486 487 488 489 490 497 498
 499 500 501 502 525 526 527 528 529 530 531], Weight = [0.13490864 0.20056036 0.25330008 0.28513933 0.29213436 0.27456125
 0.23611504 0.18261099 0.12073645 0.11227218 0.18484422 0.24092033
 0.26828681 0.2602826  0.21809638 0.15112532 0.1047817  0.16286432
 0.19923032 0.20521867 0.18049444 0.13249467 0.12004205 0.16491299
 0.19736295 0.20927639 0.19646852 0.1603138  0.10782198]
wdist = 1.8696699565298651, cdiff = 18.00603457141207
RSS = 153.70175366873292
2.614635617851434


In [23]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [469 471 472 487 488 499 500 527 528], Weight = [0.45424007 1.17786671 0.21697766 0.57416642 0.85227729 0.50602988
 0.48306009 0.45369149 0.7238494 ]
wdist = 0.604852940125768, cdiff = -0.39650806898040825
RSS = 10.01973168114615
2.774549745000627


In [24]:
with h5py.File('result/takara/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/takara.pgf')
fig.savefig('Note/figures/takara.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [470 471 472 487 488 499 500 528], Weight = [0.8683057  0.25852837 0.75238555 0.73799834 0.6444972  0.49203569
 0.50425877 1.15523982]
wdist = 0.8129154294613066, cdiff = -5.021948382194073
RSS = 19.534707730708483
2.9598339189554395


In [25]:
with h5py.File('result/mcmc/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/mcmc.pgf')
fig.savefig('Note/figures/mcmc.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [465.  467.  468.5 469.5 471.5 472.  472.5 473.  474.  475.5 476.  482.5
 483.5 486.  486.5 487.  487.5 488.5 489.  489.5 490.5 491.  492.  494.5
 495.  496.  497.  498.5 500.  525.  527.  527.5 528.  529.  529.5 530.
 530.5 531.  531.5 532. ], Weight = [0.00004056 0.00000161 0.50646792 0.00000606 1.3935919  0.00002333
 0.00002546 0.0000265  0.0000705  0.00002203 0.00000904 0.00001444
 0.00000447 0.00000808 0.00000127 0.000059   1.37724389 0.00001398
 0.00000858 0.00001346 0.00002844 0.00002556 0.00001106 0.00000025
 0.0000299  0.00003845 0.0000416  0.36676836 0.65237934 0.00000705
 0.00002421 1.15792123 0.00000763 0.00003973 0.00002968 0.0000268
 0.00001788 0.00000645 0.00002019 0.00004749]
wdist = 0.6243421832926701, cdiff = 1.6777548751523437
RSS = 8.834620517524197
2.67

In [26]:
# pet, pwe = wff.xiaopeip(wave, spe_pre[cid], eta=0)
pet, pwe = wff.xiaopeip(wave, spe_pre[cid], Tau, Sigma, Thres['lucyddm'], p, eta=0)
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(*ax1_lim)
ax2.set_ylim(*ax2_lim)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [467. 468. 470. 471. 472. 487. 488. 499. 500. 527. 528.], Weight = [0.08310591 0.00032206 0.97774912 0.01575962 0.81853973 0.59466171
 0.79408098 0.493306   0.51071231 0.32664292 0.83637863]
wdist = 0.9068197801619707, cdiff = 1.0594602473590373
RSS = 6.49430638214507
2.7278145296716048


In [27]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, norm.cdf(-1) * 1e2)
        wdist_stats[i, 5] = np.percentile(dtwpi, norm.cdf(1) * 1e2)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, norm.cdf(-1) * 1e2)
        edist_stats[i, 5] = np.percentile(dtepi, norm.cdf(1) * 1e2)

    L = len(dt)
    data = dt['wdist']
    fig = plt.figure(figsize=(8, 6))
    ax1 = fig.add_axes((.125, .12, .6, .77))
    # boxdict = ax1.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax1.plot(np.arange(1, l + 1), wdist_stats[:, 0], color='b')
    
    ax1.errorbar(np.arange(1, l + 1), 
                wdist_stats[:, 0], 
                yerr=[wdist_stats[:, 0] - wdist_stats[:, 4], wdist_stats[:, 5] - wdist_stats[:, 0]], 
                fmt='o', elinewidth=1, capsize=3, color='b', label=r'$D_\mathrm{w}$')
    
    ax1.fill_between(np.arange(1, l + 1), 
                    wdist_stats[:, 4], 
                    wdist_stats[:, 5], fc='k', alpha=0.1, color=None)

    ax1.set_xticks(np.arange(1, 16, 2))
    ax1.set_xticklabels(np.arange(1, 16, 2).astype(str))
    ax1.set_xlim(0, l + 1)
    # ax1.set_ylim(0, max([boxdict['whiskers'][2 * i + 1].get_xydata()[1, 1] for i in range(l)]) * 1.05)
    # ax1.set_ylim(0, wdist_stats[:, 5].max() * 1.05)
    ax1.set_xlabel(r'$N_{\mathrm{PE}}$')
    ax1.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
    ax1.legend()

    ax2 = fig.add_axes((.725, .12, .175, .77))
    bins = np.linspace(0, data[data < np.percentile(data, 90)].std() * 3 + np.mean(data), 41)
    ax2.hist(data, bins=bins, density=1, orientation='horizontal')

    ax2.set_xlabel(r'$\mathrm{arb.\ unit}$')
    ax2.set_xlim(0, ax2.get_xlim()[1] * 1.05)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_ylim(0, bins[-1])
    ax1.set_ylim(0, bins[-1])
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)

In [28]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
tlist = np.arange(-50, 200)

for mu in tqdm(Mu * wff.convolve_exp_norm(tlist, Tau, Sigma)):
    for i in range(1, 15):
        # pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))
        pdf += mu * poisson.pmf(i, mu) * gamma.pdf(t, a=(gmu/gsigma)**2, loc=0, scale=(gsigma/gmu)**2/np.sqrt(i))

100%|██████████| 250/250 [00:02<00:00, 106.83it/s]


In [29]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fsmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
t = np.arange(0, 1000, 0.1) / gmu
# ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fsmp':'\mathrm{FSMP}', 'fsmpwave':'\mathrm{FSMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/' + file + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist.png')
plt.savefig('Note/figures/recchargehist.pdf')
plt.savefig('Note/figures/recchargehist.pgf')
plt.show()

In [30]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
b = 0.5
tlist = np.arange(-50, 200, b)

for mu in tqdm(25 * wff.convolve_exp_norm(tlist, Tau, Sigma) * b):
    for i in range(1, 15):
        # pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))
        pdf += mu * poisson.pmf(i, mu) * gamma.pdf(t, a=(gmu/gsigma)**2, loc=0, scale=(gsigma/gmu)**2/np.sqrt(i))

100%|██████████| 500/500 [00:04<00:00, 108.78it/s]


In [31]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fsmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
# ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fsmp':'\mathrm{FSMP}', 'fsmpwave':'\mathrm{FSMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/15.0-20-5' + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist25.png')
# plt.savefig('Note/figures/recchargehist.pdf')
# plt.savefig('Note/figures/recchargehist.pgf')
plt.show()